In [9]:
import undetected_chromedriver
from selenium.webdriver.common.by import By
import re
import json

In [142]:
DRIVER = undetected_chromedriver.Chrome()
DRIVER.implicitly_wait(10)

In [4]:
DRIVER.get("https://www.parfumo.com/Users/Crazysillage/Collection/Abfuellungen")

links = []
for fragrance in DRIVER.find_element(By.ID, "wr_wrapper").find_elements(By.XPATH, "*"):
    fragrance.click()
    link = DRIVER.find_element(
        By.CSS_SELECTOR, "body > div.wr_sneak > div.header > div.img > a"
    )
    links.append(link.get_attribute("href"))

with open("decants.json", "w") as f:
    json.dump(links, f)

['https://www.parfumo.com/Perfumes/Akro/Dark', 'https://www.parfumo.com/Perfumes/Akro/Awake', 'https://www.parfumo.com/Perfumes/Les_Indemodables/vanille-havane-eau-de-parfum', 'https://www.parfumo.com/Perfumes/Widian/Sapphire_Collection__London', 'https://www.parfumo.com/Perfumes/Parfums_de_Nicolai/Ambre_Cashmere_Intense', 'https://www.parfumo.com/Perfumes/Strangers_Parfumerie/Cigar_Rum', 'https://www.parfumo.com/Perfumes/Strangers_Parfumerie/Oliver', 'https://www.parfumo.com/Perfumes/Strangers_Parfumerie/Salted_Green_Mango', 'https://www.parfumo.com/Perfumes/Rogue/bon-monsieur', 'https://www.parfumo.com/Perfumes/Rogue/targhee-forest', 'https://www.parfumo.com/Perfumes/Rogue/Mousse_Illuminee', 'https://www.parfumo.com/Perfumes/Dolce_Gabbana/Light_Blue_pour_Homme_Eau_de_Toilette', 'https://www.parfumo.com/Perfumes/Ralph_Lauren/polo-red-parfum', 'https://www.parfumo.com/Perfumes/Versace/eros-eau-de-parfum', 'https://www.parfumo.com/Perfumes/Azzaro/wanted-eau-de-parfum', 'https://www.parf

In [151]:
with open("decants.json", "r") as f:
    decants = json.load(f)

decants_enriched = []
for fragrance in decants:
    DRIVER.get(fragrance)
    DRIVER.find_element(By.ID, "classi_li").click()
    name = DRIVER.find_element(
        By.CSS_SELECTOR, "#pd_inf > div.cb.pt-1 > main > div.p_details_holder > h1"
    ).text
    brand = DRIVER.find_element(
        By.CSS_SELECTOR,
        "#pd_inf > div.cb.pt-1 > main > div.p_details_holder > h1 > span > span > a:nth-child(1) > span",
    ).text

    type_script = DRIVER.find_element(
        By.CSS_SELECTOR, "#classification_community > script:nth-child(2)"
    ).get_attribute("innerHTML")
    scent_type = json.loads(re.search("chart4\.data = ([^;]+)", type_script).group(1))

    occasion_script = DRIVER.find_element(
        By.CSS_SELECTOR, "#classification_community > script:nth-child(14)"
    ).get_attribute("innerHTML")
    scent_season = json.loads(
        re.search("chart2\.data = ([^;]+)", occasion_script).group(1)
    )

    audience_script = DRIVER.find_element(
        By.CSS_SELECTOR, "#classification_community > script:nth-child(6)"
    ).get_attribute("innerHTML")
    scent_audience = json.loads(
        re.search("chart1\.data = ([^;]+)", audience_script).group(1)
    )

    season_script = DRIVER.find_element(
        By.CSS_SELECTOR, "#classification_community > script:nth-child(10)"
    ).get_attribute("innerHTML")
    scent_season = json.loads(
        re.search("chart3\.data = ([^;]+)", season_script).group(1)
    )

    decants_enriched.append(
        {
            "name": name,
            "brand": brand,
            "link": fragrance,
            "type": scent_type,
            "occasion": scent_occasion,
            "audience": scent_audience,
            "season": scent_season,
        }
    )

with open("decants_enriched.json", "w") as f:
    json.dump(decants_enriched, f)

In [158]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from numpy import average

df = pd.read_json("decants_enriched.json")
df["name"] = df["name"].apply(lambda x: re.sub("\n\w+ ", " ", x))

type_df = df.explode("type")
type_df["type_name"] = type_df["type"].apply(lambda x: x["ct_name"])
type_df["type_votes"] = type_df["type"].apply(lambda x: x["votes"])
type_pivot = (
    type_df[["name", "type_name", "type_votes"]]
    .pivot(index="name", columns="type_name", values="type_votes")
    .fillna(0)
    .astype("int32")
)
type_pivot = type_pivot.apply(lambda row: row / row.sum(), axis=1)
type_cosine_array = cosine_similarity(type_pivot).round(3)

occasion_df = df.explode("occasion")
occasion_df["occasion_name"] = occasion_df["occasion"].apply(lambda x: x["ct_name"])
occasion_df["occasion_votes"] = occasion_df["occasion"].apply(lambda x: x["votes"])
occasion_pivot = (
    occasion_df[["name", "occasion_name", "occasion_votes"]]
    .pivot(index="name", columns="occasion_name", values="occasion_votes")
    .fillna(0)
    .astype("int32")
)
occasion_pivot = occasion_pivot.apply(lambda row: row / row.sum(), axis=1)
occasion_cosine_array = cosine_similarity(occasion_pivot).round(3)

season_df = df.explode("season")
season_df["season_name"] = season_df["season"].apply(lambda x: x["ct_name"])
season_df["season_votes"] = season_df["season"].apply(lambda x: x["votes"])
season_pivot = (
    season_df[["name", "season_name", "season_votes"]]
    .pivot(index="name", columns="season_name", values="season_votes")
    .fillna(0)
    .astype("int32")
)
season_pivot = season_pivot.apply(lambda row: row / row.sum(), axis=1)
season_cosine_array = cosine_similarity(season_pivot).round(3)

audience_df = df.explode("audience")
audience_df["audience_name"] = audience_df["audience"].apply(lambda x: x["ct_name"])
audience_df["audience_votes"] = audience_df["audience"].apply(lambda x: x["votes"])
audience_pivot = (
    audience_df[["name", "audience_name", "audience_votes"]]
    .pivot(index="name", columns="audience_name", values="audience_votes")
    .fillna(0)
    .astype("int32")
)
audience_pivot = audience_pivot.apply(lambda row: row / row.sum(), axis=1)
audience_cosine_array = cosine_similarity(audience_pivot).round(3)

closest_frags = {}
for i in range(0, len(type_cosine_array)):
    for j in range(0, i):
        closest_frags[f"{type_pivot.index[i]} / {type_pivot.index[j]}"] = (
            type_cosine_array[i][j],
            occasion_cosine_array[i][j],
            season_cosine_array[i][j],
            audience_cosine_array[i][j],
        )

weights = [0.5, 0.25, 0.25, 0.0]
for pair in sorted(
    closest_frags.items(),
    key=lambda x: average([x[1][0], x[1][1], x[1][2], x[1][3]], weights=weights),
    reverse=True,
):
    print(f"{pair[1]} : {pair[0]}")

(0.968, 0.998, 1.0, 0.847) : Emporio Armani - Stronger With You Absolutely Armani 2021 / Angels' Share 2020
(0.97, 0.995, 0.998, 0.993) : Chrome 1996 EAU DE TOILETTE / Aqva pour Homme 2005 EAU DE TOILETTE
(0.955, 0.998, 0.997, 0.868) : Terre d'Hermès Eau Givrée 2022 / Bergamote 22 Labo 2006 EAU DE PARFUM
(0.946, 0.998, 0.994, 0.973) : Gentleman Givenchy Réserve Privée 2022 / Dior Homme Intense 2011
(0.953, 0.985, 0.983, 0.978) : Aqva pour Homme 2005 EAU DE TOILETTE / Acqua di Giò Parfum Armani 2023
(0.938, 0.993, 0.994, 0.998) : Ultra Mâle Paul Gaultier 2015 / 1 Million Elixir Rabanne 2022
(0.93, 0.985, 0.996, 0.981) : Oud for Greatness 2018 / Haltane de Marly 2021
(0.935, 0.984, 0.981, 0.985) : № 02 - L'Air du Désert Marocain Perfumes 2005 EAU DE TOILETTE INTENSE / Oud for Greatness 2018
(0.917, 0.997, 0.999, 0.999) : Legend 2011 EAU DE TOILETTE / Explorer 2019
(0.923, 0.99, 0.994, 0.966) : Myslf Saint Laurent 2023 / Legend 2020 EAU DE PARFUM
(0.949, 0.957, 0.971, 0.979) : Light Blue 